In [17]:
import asyncio
import httpx
import time

# The endpoint for your local container. If you used the nginx version, use port 80; else port 8000.
BASE_URL = "http://localhost/predict"

In [18]:
# Define a sample text payload
sample_payload = {"text": "I love deploying FastAPI in Docker!"}

# Synchronous test (for comparison)
start_time = time.time()
response = httpx.post(BASE_URL, json=sample_payload)
end_time = time.time()
print("Status:", response.status_code)
print("Response JSON:", response.json())
print(f"Completed 1 request in {end_time - start_time:.2f} seconds.\n")

Status: 200
Response JSON: {'label': 'POSITIVE', 'score': 0.9872898459434509}
Completed 1 request in 2.79 seconds.



In [19]:
async def fetch_sentiment(client: httpx.AsyncClient, text: str):
    payload = {"text": text}
    try:
        resp = await client.post(BASE_URL, json=payload, timeout=10.0)
        return resp.json()
    except Exception as e:
        return {"error": str(e)}

In [20]:
async def run_parallel_requests(texts: list[str]):
    async with httpx.AsyncClient() as client:
        tasks = []
        for t in texts:
            tasks.append(fetch_sentiment(client, t))
        # Gather results concurrently
        results = await asyncio.gather(*tasks)
    return results

In [21]:
test_texts = [
    "I love this product!",
    "This is the worst experience I've ever had.",
    "FastAPI and Docker are awesome.",
    "I hate waiting on slow APIs.",
    "The weather is nice today.",
    "I am not sure about this model.",
    "Parallel requests are fun to demo.",
    "Error handling is important.",
    "Let's go to the movies tonight.",
    "I didn't like the service.",
    "Absolutely fantastic!",
    "Terrible, I want a refund.",
    "I'll probably recommend this to friends.",
    "This is okay, not great.",
    "Innovation drives success.",
    "I'm bored right now.",
    "Looking forward to the weekend.",
    "I'm upset with the delay.",
    "They delivered quickly.",
    "Could be better, could be worse."
]

# Time the parallel execution
start_time = time.time()
results = await run_parallel_requests(test_texts)
end_time = time.time()

print(f"Completed {len(test_texts)} requests in {end_time - start_time:.2f} seconds.\n")
for i, res in enumerate(results, 1):
    print(f"{i:2d}. Input: '{test_texts[i-1]}' → {res}")

Completed 20 requests in 5.54 seconds.

 1. Input: 'I love this product!' → {'label': 'POSITIVE', 'score': 0.9998855590820312}
 2. Input: 'This is the worst experience I've ever had.' → {'label': 'NEGATIVE', 'score': 0.9997679591178894}
 3. Input: 'FastAPI and Docker are awesome.' → {'label': 'POSITIVE', 'score': 0.9998502731323242}
 4. Input: 'I hate waiting on slow APIs.' → {'label': 'NEGATIVE', 'score': 0.996658444404602}
 5. Input: 'The weather is nice today.' → {'label': 'POSITIVE', 'score': 0.9998351335525513}
 6. Input: 'I am not sure about this model.' → {'label': 'NEGATIVE', 'score': 0.9996001124382019}
 7. Input: 'Parallel requests are fun to demo.' → {'label': 'POSITIVE', 'score': 0.9964538812637329}
 8. Input: 'Error handling is important.' → {'label': 'POSITIVE', 'score': 0.8825832605361938}
 9. Input: 'Let's go to the movies tonight.' → {'label': 'POSITIVE', 'score': 0.9969881176948547}
10. Input: 'I didn't like the service.' → {'label': 'NEGATIVE', 'score': 0.99694257974